Load the stock_prediction_data.csv and stock_price.csv
stock_prediction_data.csv contains the data of stock prices and world events
stock_price.csv contains the difference in stock price the next day.
Split the data into Train, validation, test
Preprocess the data, remove center and scale it. 

Perform 2nd order polynomial regression with
Lasso constraint
Solve it with gradient descent
Solve it with sklearn library (compare your results)
Use the validation data to identify and ideal lambda value

Ridge constraint
Solve it with gradient descent
Solve it with sklearn library (compare your results)
Use the validation data to identify and ideal lambda value


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

stock_prediction_data = pd.read_csv('/Users/shreyas/Desktop/ML/HW/hw3/stock_prediction_data.csv')
stock_price_data = pd.read_csv('/Users/shreyas/Desktop/ML/HW/hw3/stock_price.csv')
X = stock_prediction_data.values
y = stock_price_data.values.ravel()


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_val_poly = poly.transform(X_val_scaled)
X_test_poly = poly.transform(X_test_scaled)

class LassoRegressionGD:
    def __init__(self, lr=0.001, epochs=1000, lambda_=1):
        self.lr = lr
        self.epochs = epochs
        self.lambda_ = lambda_

    def fit(self, X, y):
        self.m, self.n = X.shape
        self.theta = np.zeros(self.n)
        for _ in range(self.epochs):
            gradients = (2/self.m) * X.T.dot(X.dot(self.theta) - y) + self.lambda_ * np.sign(self.theta)
            self.theta -= self.lr * gradients

    def predict(self, X):
        return X.dot(self.theta)

# Lasso using Gradient Descent
lasso_gd = LassoRegressionGD(lr=0.001, epochs=1000, lambda_=0.1)
lasso_gd.fit(X_train_poly, y_train)
y_val_pred_lasso_gd = lasso_gd.predict(X_val_poly)
mse_lasso_gd = mean_squared_error(y_val, y_val_pred_lasso_gd)
print(f"Lasso with Gradient Descent MSE: {mse_lasso_gd}")

# Lasso using Sklearn
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_poly, y_train)
y_val_pred_lasso = lasso.predict(X_val_poly)
mse_lasso_sklearn = mean_squared_error(y_val, y_val_pred_lasso)
print(f"Lasso with Sklearn MSE: {mse_lasso_sklearn}")

# Ridge (L2 Regularization) with Gradient Descent
class RidgeRegressionGD:
    def __init__(self, lr=0.001, epochs=1000, lambda_=1):
        self.lr = lr
        self.epochs = epochs
        self.lambda_ = lambda_

    def fit(self, X, y):
        self.m, self.n = X.shape
        self.theta = np.zeros(self.n)
        for _ in range(self.epochs):
            gradients = (2/self.m) * X.T.dot(X.dot(self.theta) - y) + 2 * self.lambda_ * self.theta
            self.theta -= self.lr * gradients

    def predict(self, X):
        return X.dot(self.theta)

# Ridge using Gradient Descent
ridge_gd = RidgeRegressionGD(lr=0.001, epochs=1000, lambda_=0.1)
ridge_gd.fit(X_train_poly, y_train)
y_val_pred_ridge_gd = ridge_gd.predict(X_val_poly)
mse_ridge_gd = mean_squared_error(y_val, y_val_pred_ridge_gd)
print(f"Ridge with Gradient Descent MSE: {mse_ridge_gd}")

# Ridge using Sklearn
ridge = Ridge(alpha=0.1)
ridge.fit(X_train_poly, y_train)
y_val_pred_ridge = ridge.predict(X_val_poly)
mse_ridge_sklearn = mean_squared_error(y_val, y_val_pred_ridge)
print(f"Ridge with Sklearn MSE: {mse_ridge_sklearn}")

# Tuning Lambda for Lasso using Grid Search
lasso_params = {'alpha': np.logspace(-4, 4, 50)}
lasso_grid = GridSearchCV(Lasso(), lasso_params, scoring='neg_mean_squared_error', cv=5)
lasso_grid.fit(X_train_poly, y_train)
best_lasso = lasso_grid.best_estimator_
print(f"Best Lasso Lambda: {best_lasso.get_params()['alpha']}")

# Tuning Lambda for Ridge using Grid Search
ridge_params = {'alpha': np.logspace(-4, 4, 50)}
ridge_grid = GridSearchCV(Ridge(), ridge_params, scoring='neg_mean_squared_error', cv=5)
ridge_grid.fit(X_train_poly, y_train)
best_ridge = ridge_grid.best_estimator_
print(f"Best Ridge Lambda: {best_ridge.get_params()['alpha']}")



Lasso with Gradient Descent MSE: 3.724652711715657
Lasso with Sklearn MSE: 0.1233903914225661
Ridge with Gradient Descent MSE: 5.674331757210358
Ridge with Sklearn MSE: 0.0645473832205928
Best Lasso Lambda: 0.019306977288832496
Best Ridge Lambda: 0.18420699693267145
